In [1]:
import os
from utils.file import get_all_files_in_dir_with_format, check_folder_exists_or_file_exists, copy_folder, create_file
from utils.spliter import split_dataset
from utils.converter import convert_to_yolo
from utils.yolo import create_yolo_yaml_config, YOLOYamlConfig

In [2]:
BASE_PATH = os.getcwd()
ORIGINAL_DATA_PATH = os.path.join(os.path.dirname(BASE_PATH), 'data/original')
ORIGINAL_ANNOTATIONS_PATH = os.path.join(ORIGINAL_DATA_PATH, 'Bounding_boxes')
ORIGINAL_IMAGES_PATH = os.path.join(ORIGINAL_DATA_PATH, 'images')
OUTPUT_DIR = os.path.join(ORIGINAL_DATA_PATH, 'yolo_annotations')

CONVERTED_DATA_PATH = os.path.join(os.path.dirname(BASE_PATH), 'data/converted')
CONVERTED_ANNOTATIONS_PATH = os.path.join(CONVERTED_DATA_PATH, 'labels')
CONVERTED_TRAIN_TEST_PATH = [os.path.join(CONVERTED_ANNOTATIONS_PATH, 'train'), os.path.join(CONVERTED_ANNOTATIONS_PATH, 'test')]
CONVERTED_IMAGES_PATH = os.path.join(CONVERTED_DATA_PATH, 'images')

FINAL_DATA_PATH = os.path.join(os.path.dirname(BASE_PATH), 'data/final')
YOLO_CONFIG_PATH = os.path.join(os.path.dirname(BASE_PATH), 'core/packages/yolov5/data/my_yolo_config.yaml')


In [5]:
annotations_files = get_all_files_in_dir_with_format(ORIGINAL_ANNOTATIONS_PATH, ('.csv',))
for key, value in enumerate(annotations_files):
    if not check_folder_exists_or_file_exists(CONVERTED_TRAIN_TEST_PATH[key]):
        convert_to_yolo(value, CONVERTED_TRAIN_TEST_PATH[key])

copy_folder(ORIGINAL_IMAGES_PATH, CONVERTED_IMAGES_PATH)

Folder /Users/cobeo/Codes/SchoolWork/COS40007/Assignment5/data/converted/images already exists. Skipping copy.


In [6]:
train_files, test_files, val_files = split_dataset(CONVERTED_DATA_PATH, FINAL_DATA_PATH)

Found 813 training images with matching labels
Found 209 test images with matching labels
Dataset split complete:
  - Training: 400 images
  - Testing: 40 images
  - Validation: 40 images


In [12]:
if not check_folder_exists_or_file_exists(YOLO_CONFIG_PATH):
    create_file(YOLO_CONFIG_PATH)

configurator = YOLOYamlConfig(
  path=YOLO_CONFIG_PATH,
  root_dir_path=FINAL_DATA_PATH,
  train_path="images/train",
  val_path="images/val",
  test_path="images/test",
  names={
    0: 'Graffiti',
  }
)

create_yolo_yaml_config(configurator)

In [ ]:
!python packages/yolov5/train.py --device mps --data packages/yolov5/data/my_yolo_config.yaml --weights yolov5m.pt --epochs 20 --batch-size 16 --name iteration1  # from pretrained (recommended)